## About
This notebook contains simple, toy examples to help you get started with FairMLHealth tool usage. This same content is mirrored in the repository's main [README](../README.md).

## Example Setup

In [1]:
from fairmlhealth import report, measure, stat_utils

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


In [2]:
# First we'll create a semi-randomized dataframe with specific columns for our attributes of interest
np.random.seed(506)
N = 240
X = pd.DataFrame({'col1': np.random.randint(1, 4, N), 
                  'col2': np.random.randint(1, 75, N),
                  'col3': np.random.randint(0, 2, N),
                  'gender': [0, 1]*int(N/2), 
                  'ethnicity': [1, 1, 0, 0]*int(N/4),
                  'other': [1, 0, 0, 0, 1, 0, 0, 1]*int(N/8)
                 })

# Next we'll create a randomized target value
y = pd.Series(X['col3'].values + np.random.randint(0, 2, N), name='Example_Target').clip(upper=1)

# Third, we'll split the data and use it to train two generic models
splits = train_test_split(X, y, stratify=y, test_size=0.5, random_state=60)
X_train, X_test, y_train, y_test = splits

model_1 = BernoulliNB().fit(X_train, y_train)
model_2 = DecisionTreeClassifier().fit(X_train, y_train)



In [3]:
display(X.head(), y.head())

,col1,col2,col3,gender,ethnicity,other
0,1,15,0,0,1,1
1,3,51,1,1,1,0
2,1,30,1,0,0,0
3,2,28,1,1,0,0
4,1,72,0,0,1,1


0    0
1    1
2    1
3    1
4    1
Name: Example_Target, dtype: int64

## Generalized Reports
fairMLHealth has tools to create generalized reports of model bias and performance.

The primary reporting tool is now the **compare** function, which can be used to generate side-by-side comparisons for any number of models, and for either binary classifcation or for regression problems. Model performance metrics such as accuracy and precision (or MAE and RSquared for regression problems) are also provided to facilitate comparison. Below is an example output comparing the two example models defined above. Missing values have been added for metrics requiring prediction probabilities (which the second model does not have).

A flagging protocol is applied by default to highlight any cells with values that are out of range.  This can be turned off by passing ***flag_oor = False*** to report.compare().

*Note that the Equal Odds Ratio has been dropped from the example below*. This because the false positive rate is approximately zero for both the entire dataset and for the privileged class, leading to a zero in the denominator of the False Positive Rate Ratio: $\frac{{FPR}_{unprivileged}}{{FPR}_{privileged}}$. The result is therefore undefined and cannot be compared in the Equal Odds Ratio. 

In [4]:
# Generate a measure report
report.compare(X_test, y_test, X_test['ethnicity'], model_1, flag_oor=True)


~/fairMLHealth/fairmlhealth/measure.py:713: UserWarning: The following measures are undefined and have been dropped: ['Equal Odds Ratio']
  warn(f"The following measures are undefined and have been dropped: {undefined}")


In [5]:
# Return the report as embedded html
from IPython.core.display import HTML
html_output = report.compare(X_test, y_test, X_test['gender'], model_1, pred_type="classification", output_type="html")
HTML(html_output)

### Comparing Results for Multiple Models

The **compare** tool can also be used to measure two different models or two different protected attributes. Protected attributes are measured separately and cannot yet be combined together with the **compare** tool, although they can be grouped as cohorts in the stratified tables [as shown below](#cohort). 

Below is an example output comparing the two test models defined above. 

In [6]:
# Example with multiple models
report.compare(X_test, y_test, X_test['gender'],
               {'model 1':model_1, 'model 2':model_2})

In [7]:
# Example with different protected attributes. 
# Note that the same model is passed with two different keys to clarify the column names.
report.compare(X_test, y_test, 
                    [X_test['gender'], X_test['ethnicity']], 
                    {'gender':model_1, 'ethnicity':model_1},
                    flag_oor=True)


~/fairMLHealth/fairmlhealth/measure.py:713: UserWarning: The following measures are undefined and have been dropped: ['Equal Odds Ratio']
  warn(f"The following measures are undefined and have been dropped: {undefined}")


## Detailed Analyses


### Significance Testing

It's recommended to test for the statistical significance of discrepancies in the distribution of results. This is particularly true for attributes with skewed distributions, for which small sample sizes for less common labels may affect fairness measures. However it is still generally recommended to validate whether the test sample for which fairness measures are generated is reflective of the full dataset.

FairMLHealth comes with a bootstrapping utility and supporting funcitons that can be used in some statistical testing. While the selection of proper statistical tests is beyond the scope of this notebooks, two examples using the bootstrap_significance tool with built-in test functions are shown below.

In [8]:
model_1_preds = pd.Series(model_1.predict(X_test))

In [9]:
# Example Significance Test Results Applying Kruskal-Wallis to Predictions
isMale = X_test.reset_index(drop=True)['gender'].eq(1)
reject_h0 = stat_utils.bootstrap_significance(alpha=0.05,
                                              func=stat_utils.kruskal_pval, 
                                              dist_a=model_1_preds.loc[isMale], 
                                              dist_b=model_1_preds.loc[~isMale])
print("Is it likely that the difference in the mean y value is related to gender?\n",
      reject_h0)

Is it likely that the difference in the mean y value is related to gender?
 False


In [10]:
# Example Significance Test Results Applying Chi-Square to the Distribution of Prediction Successes/Failures
model_1_results = stat_utils.binary_result_labels(y_test, model_1_preds)
reject_h0 = stat_utils.bootstrap_significance(alpha=0.05,
                                              func=stat_utils.chisquare_pval, 
                                              group=X_test['gender'], 
                                              values=model_1_results)
print("Can we reject the hypothesis that prediction results are from the same", 
      "distribution for each gender?\n", reject_h0)

Can we reject the hypothesis that prediction results are from the same distribution for each gender?
 False


### Stratified Tables
FairMLHealth also provides tools for detailed analysis of model variance by way of stratified data, performance, and bias tables. Beyond evaluating fairness, these tools are intended for flexible use in any generic assessment of model bais. Tables can evaluate multiple features at once. *An important update starting in Version 1.0.0 is that all of these features are now contained in the **measure.py** module (previously named reports.py).*

All tables display a summary row for "All Features, All Values". This summary can be turned off by passing ***add_overview=False*** to measure.data().



#### Data Tables

The stratified data table can be used to evaluate data against one or multiple targets. Two methods are available for identifying which features to assess, as shown in the examples below. 


In [11]:
# Arguments Option 1: pass full set of data, subsetting with *features* argument
measure.data(X_test, y_test, features=['gender', 'other', 'col1'])

,Feature Name,Feature Value,Obs.,Entropy,Mean Example_Target,Median Example_Target,Missing Values,Std. Dev. Example_Target,Value Prevalence
0,ALL FEATURES,ALL VALUES,120,NaN,0.7500,1.0,0,0.4348,1.0000
1,gender,0,61,0.9998,0.7213,1.0,0,0.4521,0.5083
2,gender,1,59,0.9998,0.7797,1.0,0,0.4180,0.4917
3,other,0,77,0.9413,0.7922,1.0,0,0.4084,0.6417
4,other,1,43,0.9413,0.6744,1.0,0,0.4741,0.3583
5,col1,1,42,1.5838,0.7857,1.0,0,0.4153,0.3500
6,col1,2,40,1.5838,0.8250,1.0,0,0.3848,0.3333
7,col1,3,38,1.5838,0.6316,1.0,0,0.4889,0.3167


In [12]:
# Arguments Option 2: pass the data subset of interest without using the *features* argument
measure.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean col3,Median col2,Median col3,Missing Values,Std. Dev. col2,Std. Dev. col3,Value Prevalence
0,ALL FEATURES,ALL VALUES,120,NaN,39.3167,0.5083,40.5,1.0,0,20.7543,0.5020,1.0000
1,gender,0,61,0.9998,39.7705,0.4098,39.0,0.0,0,21.6482,0.4959,0.5083
2,gender,1,59,0.9998,38.8475,0.6102,41.0,1.0,0,19.9627,0.4919,0.4917
3,col1,1,42,1.5838,38.5000,0.5952,39.5,1.0,0,20.6424,0.4968,0.3500
4,col1,2,40,1.5838,35.7250,0.5250,34.0,1.0,0,18.2897,0.5057,0.3333
5,col1,3,38,1.5838,44.0000,0.3947,49.5,0.0,0,22.8769,0.4954,0.3167


In [13]:
# Display a similar report for multiple targets, dropping the summary row
measure.data(X=X_test, # used to define rows
             Y=X_test, # used to define columns
             features=['gender', 'col1'], # optional subset of X
             targets=['col2', 'col3'], # optional subset of Y
             add_overview=False # turns off "All Features, All Values" row
             )

,Feature Name,Feature Value,Obs.,Entropy,Mean col2,Mean col3,Median col2,Median col3,Missing Values,Std. Dev. col2,Std. Dev. col3,Value Prevalence
0,gender,0,61,0.9998,39.7705,0.4098,39.0,0.0,0,21.6482,0.4959,0.5083
1,gender,1,59,0.9998,38.8475,0.6102,41.0,1.0,0,19.9627,0.4919,0.4917
2,col1,1,42,1.5838,38.5000,0.5952,39.5,1.0,0,20.6424,0.4968,0.3500
3,col1,2,40,1.5838,35.7250,0.5250,34.0,1.0,0,18.2897,0.5057,0.3333
4,col1,3,38,1.5838,44.0000,0.3947,49.5,0.0,0,22.8769,0.4954,0.3167


#### Stratified Performance Tables

The stratified performance table evaluates model performance specific to each feature-value subset. These tables are compatible with both classification and regression models. For classification models with the *predict_proba()* method, additional ROC_AUC and PR_AUC values will be included if possible.

In [14]:
# Performance table example
measure.performance(X_test[['gender']], y_test, model_1.predict(X_test))

,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,Precision,TPR
0,ALL FEATURES,ALL VALUES,120.0,0.7500,0.7750,0.7750,0.8525,0.5000,0.8387,0.8667
1,gender,0,61.0,0.7213,0.7377,0.7869,0.8539,0.4118,0.8444,0.8636
2,gender,1,59.0,0.7797,0.8136,0.7627,0.8511,0.6154,0.8333,0.8696


In [15]:
# Performance table example with probabilities included
measure.performance(X_test[['gender']], 
                    y_true=y_test, 
                    y_pred=model_1.predict(X_test), 
                    y_prob=model_1.predict_proba(X_test)[:,1])

,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,120.0,0.7500,0.7750,0.7750,0.8525,0.5000,0.2062,0.8387,0.8583,0.8667
1,gender,0,61.0,0.7213,0.7377,0.7869,0.8539,0.4118,0.2261,0.8444,0.8429,0.8636
2,gender,1,59.0,0.7797,0.8136,0.7627,0.8511,0.6154,0.1873,0.8333,0.8679,0.8696


#### Stratified Bias Tables

The stratified bias analysis table apply fairness-related metrics for each feature-value pair. It assumes a given feature-value as the "privileged" group relative to all other possible values for the feature. For example, row **2** in the table below displays measures for **"col1"** with a value of **"2"**. For this row, "2" is considered to be the privileged group, while all other non-null values (namely "1" and "3") are considered unprivileged.

To simplify the table, fairness measures have been reduced to their component parts. For example, the Equal Odds Ratio has been reduced to the True Positive Rate (TPR) Ratio and False Positive Rate (FPR) Ratio.

Note that the *flag* function is compatible with both **measure.bias()** and **measure.summary()** (which is demonstrated below). However, to enable colored cells the tool returns a pandas Styler rather than a DataTable. For this reason, *flag_oor* is set to False by default (as shown in the example above). Flagging can be turned on by passing *flag_oor=True* to either function. As an added feature, optional custom ranges can be passed to either **measure.bias()** or **measure.summary()** to facilitate regression evaluation, shown in [Example-ToolUsage_Regression](./examples_and_tutorials/Example-ToolUsage_Regression.ipynb).

In [16]:
# Example of bias table with flag turned on
measure.bias(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), flag_oor=True)

,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,gender,0,-0.0988,0.8638,0.2036,1.4945,-0.0111,0.9868,0.0759,1.1028,0.0059,1.0069
1,gender,1,0.0988,1.1576,-0.2036,0.6691,0.0111,1.0133,-0.0759,0.9068,-0.0059,0.9932
2,col3,0,0.4569,1.8413,-0.5000,0.0000,0.4688,1.8824,0.4576,1.8438,0.4138,1.7059
3,col3,1,-0.4569,0.5431,0.5000,nan,-0.4688,0.5312,-0.4576,0.5424,-0.4138,0.5862


The **measure** module also contains a summary function that works similarly to report.compare(). While it can only be applied to one model at a time, it can accept custom "fair" ranges, and accept cohort groups as will be [shown in the next section](#cohort).

In [17]:
# Example summary with performance skipped
measure.summary(X_test[['col2']], 
                y_test, 
                model_1.predict(X_test),
                prtc_attr=X_test['gender'], 
                pred_type="classification",
                skip_performance=True
               )

Value
Metric              Measure                                       
Group Fairness      Balanced Accuracy Difference            0.0988
                    Balanced Accuracy Ratio                 1.1576
                    Statistical Parity Difference          -0.0759
                    Disparate Impact Ratio                  0.9068
                    Positive Predictive Parity Difference   0.0111
                    Positive Predictive Parity Ratio        1.0133
                    Equal Odds Difference                  -0.2036
                    Equal Odds Ratio                        0.6691
Individual Fairness Consistency Score                       0.7350
                    Between-Group Gen. Entropy Error        0.0000
Data Metrics        Prevalence of Privileged Class (%)     75.0000

## <a name="cohort"></a>Analysis by Cohort

Table-generating functions in the **measure** module can all be additionally grouped using the *cohorts* argument to specify additional labels for each observation. Cohorts may consist of either a single label or a set of labels, and may be either separate from or attached to the existing data.

In [18]:
# Separate, Single-Level Cohorts
cohort_labels = X_test['gender']
measure.bias(X_test['col3'], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=cohort_labels)

,gender,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,col3,0,0.3638,1.5718,-0.4118,0.0000,0.3500,1.5385,0.4444,1.8000,0.3158,1.4615
1,0,col3,1,-0.3638,0.6362,0.4118,nan,-0.3500,0.6500,-0.4444,0.5556,-0.3158,0.6842
2,1,col3,0,0.6077,2.5490,-0.6154,0.0000,0.6667,3.0000,0.4783,1.9167,0.6000,2.5000
3,1,col3,1,-0.6077,0.3923,0.6154,nan,-0.6667,0.3333,-0.4783,0.5217,-0.6000,0.4000


In [19]:
## Associated, Multi-Level Cohorts
measure.data(X=X_test['col3'], Y=y_test, cohorts=X_test[['gender', 'ethnicity']])

Feature Name Feature Value  Obs.  Entropy  \
gender ethnicity                                              
0      0          ALL FEATURES    ALL VALUES    32      NaN   
       0                  col3             0    20   0.9544   
       0                  col3             1    12   0.9544   
       1          ALL FEATURES    ALL VALUES    29      NaN   
       1                  col3             0    16   0.9923   
       1                  col3             1    13   0.9923   
1      0          ALL FEATURES    ALL VALUES    28      NaN   
       0                  col3             0    12   0.9852   
       0                  col3             1    16   0.9852   
       1          ALL FEATURES    ALL VALUES    31      NaN   
       1                  col3             0    11   0.9383   
       1                  col3             1    20   0.9383   

                  Mean Example_Target  Median Example_Target  Missing Values  \
gender ethnicity                                                               
0      0                       0.7812                    1.0               0   
       0                       0.6500                    1.0               0   
       0                       1.0000                    1.0               0   
       1                       0.6552                    1.0               0   
       1                       0.3750                    0.0               0   
       1                       1.0000                    1.0               0   
1      0                       0.7143                    1.0               0   
       0                       0.3333                    0.0               0   
       0                       1.0000                    1.0               0   
       1                       0.8387                    1.0               0   
       1                       0.5455                    1.0               0   
       1                       1.0000                    1.0               0   

                  Std. Dev. Example_Target  Value Prevalence  
gender ethnicity                                              
0      0                            0.4200            1.0000  
       0                            0.4894            0.6250  
       0                            0.0000            0.3750  
       1                            0.4837            1.0000  
       1                            0.5000            0.5517  
       1                            0.0000            0.4483  
1      0                            0.4600            1.0000  
       0                            0.4924            0.4286  
       0                            0.0000            0.5714  
       1                            0.3739            1.0000  
       1                            0.5222            0.3548  
       1                            0.0000            0.6452

In [20]:
# Cohorts for summary tables
measure.summary(X_test[['col2']], 
                y_test, 
                model_1.predict(X_test),
                prtc_attr=X_test['gender'], 
                pred_type="classification",
                flag_oor=False,
                skip_performance=True,
                cohorts=X_test[['ethnicity', 'col3']]
               )

/Users/christineallen/opt/anaconda3/envs/fairml/lib/python3.8/site-packages/aif360/sklearn/metrics/metrics.py:114: UndefinedMetricWarning: The ratio is ill-defined and being set to 0.0 because 'predmean' for privileged samples is 0.
  warnings.warn("The ratio is ill-defined and being set to 0.0 because "
~/fairMLHealth/fairmlhealth/measure.py:713: UserWarning: The following measures are undefined and have been dropped: ['Positive Predictive Parity Ratio', 'Equal Odds Ratio']
  warn(f"The following measures are undefined and have been dropped: {undefined}")
~/fairMLHealth/fairmlhealth/__utils.py:188: UserWarning: Could not evaluate function for group(s): {errant_list}. This is commonly caused when there is  too little data or there is only a single feature-value pair is available in a given cohort. Each cohort must have  5 observations.
  warn(msg)


Value
Metric              Measure                               ethnicity col3         
Group Fairness      Balanced Accuracy Difference          0         0      0.0000
                    Balanced Accuracy Ratio               0         0      1.0000
                    Statistical Parity Difference         0         0      0.0000
                    Disparate Impact Ratio                0         0      1.0000
                    Positive Predictive Parity Difference 0         0      0.3167
                    Positive Predictive Parity Ratio      0         0      1.9500
                    Equal Odds Difference                 0         0      0.0000
                    Equal Odds Ratio                      0         0      1.0000
Individual Fairness Consistency Score                     0         0      1.0000
                    Between-Group Gen. Entropy Error      0         0      0.0054
Data Metrics        Prevalence of Privileged Class (%)    0         0     53.0000
Group Fairness      Balanced Accuracy Difference          1         0      0.0000
                    Balanced Accuracy Ratio               1         0      1.0000
                    Statistical Parity Difference         1         0      0.0000
                    Disparate Impact Ratio                1         0      0.0000
                    Positive Predictive Parity Difference 1         0      0.0000
                    Equal Odds Difference                 1         0      0.0000
Individual Fairness Consistency Score                     1         0      1.0000
                    Between-Group Gen. Entropy Error      1         0      0.0114
Data Metrics        Prevalence of Privileged Class (%)    1         0     44.0000